In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [2]:
df_festival = pd.read_csv('./data/festivals.csv', keep_default_na=False)
df_festival.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...


In [3]:
df_film = pd.read_csv('./data/films.csv', keep_default_na=False)
df_film.head()

,id,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,1985,N/A,17 May 1985,115 min,"Drama, History, War",Youssef Chahine,"Youssef Chahine, Yusri Nasrullah","Michel Piccoli, Mohsen Mohieddin, Patrice Chér...",...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,1984,R,21 Dec 1984,120 min,"Drama, War",Alan Parker,"William Wharton (based on the novel by), Sandy...","Matthew Modine, Nicolas Cage, John Harkins, Sa...",...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,1985,R,21 Feb 1986,112 min,"Comedy, Drama",Ray Lawrence,"Ray Lawrence (screenplay), Peter Carey (screen...","Barry Otto, Lynette Curran, Helen Jones, Miles...",...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,3,,,,,,,,,,...,,,,False,,,,Movie not found!,,
4,4,The Coca-Cola Kid,1985,R,14 Jul 1985,98 min,"Comedy, Drama",Dusan Makavejev,"Frank Moorhouse (screenplay by), Denny Lawrenc...","Eric Roberts, Greta Scacchi, Bill Kerr, Chris ...",...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,


In [4]:
df = df_festival.merge(df_film, on=['id'], how='inner')
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,3,Chicken with Vinegar,Poulet au vinaigre,Claude Chabrol,France,0,cannes,1985,https://en.wikipedia.org/wiki/Chicken_with_Vin...,https://en.wikipedia.org/wiki/Claude_Chabrol,...,,,,False,,,,Movie not found!,,
4,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,


In [5]:
df = df[~df['Plot'].isin(['', 'N/A'])].reset_index(drop=True)
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,BoxOffice,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,N/A,True,6.4/10,N/A,N/A,,,
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,N/A,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,N/A,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,
3,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,N/A,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,
4,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...,...,N/A,N/A,N/A,True,7.6/10,N/A,N/A,,,


In [6]:
plots = df['Plot']
plots.head()

0    A story during the French Occupation of Egypt ...
1    Philly boys Al and Birdy became friends in hig...
2    An advertising executive dies and goes to hell...
3    An eccentric marketing guru visits a Coca-Cola...
4    Set during the fading glory of the Austro-Hung...
Name: Plot, dtype: object

### preprocessing

In [7]:
plots = [plot.lower() for plot in plots] # lowercase
plots = [re.sub('[!#$\(\)*+,./:;<=>?@^_`|~]', '', plot) for plot in plots] # remove punctuation
plots = [re.sub('\w*\d\w*', '', plot) for plot in plots] # remove numbers and words that contain numbers
plots = [re.sub('-', ' ', plot) for plot in plots] # handle compound words
plots = [word_tokenize(plot) for plot in plots] # tokenize
stopwords = set(stopwords.words('english'))
stopwords.update({'``', '\'\'', '\'s', 'n\'t', '\''})
fdist = FreqDist([word for plot in plots for word in plot])
commonwords = [word[0] for word in fdist.most_common(1000)] # try 1000
stopwords.update(commonwords) # remove most common words in corpus of film plots
lemmatizer = WordNetLemmatizer()
plots = [
    [lemmatizer.lemmatize(word) for word in plot if word not in stopwords] # remove stopwords
    for plot in plots
]
df['bag_of_words'] = plots
df.head()

,id,title_english,title_original,director,country,winner,festival,year_festival,link_film,link_director,...,Production,Website,Response,InternetMovieDatabaseRating,RottenTomatoesRating,MetacriticRating,Error,totalSeasons,Ratings,bag_of_words
0,0,Adieu Bonaparte,وداعا بونابرت,Youssef Chahine,Egypt,0,cannes,1985,https://en.wikipedia.org/wiki/Adieu_Bonaparte,https://en.wikipedia.org/wiki/Youssef_Chahine,...,N/A,N/A,True,6.4/10,N/A,N/A,,,,"[occupation, egypt, depicting, tradintion, mod..."
1,1,Birdy,N/A,Alan Parker,United States,0,cannes,1985,https://en.wikipedia.org/wiki/Birdy_(film),https://en.wikipedia.org/wiki/Alan_Parker,...,Sony Pictures Home Entertainment,N/A,True,7.3/10,85%,N/A,,,,"[philly, al, birdy, extreme, difference, perso..."
2,2,Bliss,N/A,Ray Lawrence,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/Bliss_(1985_film),https://en.wikipedia.org/wiki/Ray_Lawrence_(fi...,...,Starmaker Entertainment,N/A,True,6.9/10,N/A,N/A,,,,"[advertising, executive, hell, except, buying,..."
3,4,The Coca-Cola Kid,N/A,Dušan Makavejev,Australia,0,cannes,1985,https://en.wikipedia.org/wiki/The_Coca-Cola_Kid,https://en.wikipedia.org/wiki/Du%C5%A1an_Makav...,...,Cinecom Pictures,N/A,True,6.0/10,44%,N/A,,,,"[eccentric, marketing, guru, coca, cola, subsi..."
4,5,Colonel Redl,Oberst Redl,István Szabó,Hungary,0,cannes,1985,https://en.wikipedia.org/wiki/Colonel_Redl,https://en.wikipedia.org/wiki/Istv%C3%A1n_Szab...,...,N/A,N/A,True,7.6/10,N/A,N/A,,,,"[fading, glory, austro, hungarian, empire, ris..."


### LDA

In [8]:
dictionary = Dictionary(df['bag_of_words'])
dictionary.filter_extremes(no_below = 10) # try other values for no_below
lda_corpus = df['bag_of_words'].apply(dictionary.doc2bow)
print(dictionary)

Dictionary(1777 unique tokens: ['occupation', 'al', 'assist', 'authority', 'bird']...)


In [9]:
lda = LdaModel(corpus=lda_corpus, id2word=dictionary, num_topics=15)

In [10]:
lda.print_topics()

[(0,
  '0.011*"nick" + 0.010*"dancer" + 0.009*"elena" + 0.008*"antoine" + 0.008*"brutal" + 0.007*"dance" + 0.007*"account" + 0.007*"talent" + 0.007*"gain" + 0.006*"camille"'),
 (1,
  '0.011*"mine" + 0.009*"sicily" + 0.008*"bruno" + 0.007*"thomas" + 0.006*"order" + 0.006*"divorce" + 0.006*"miner" + 0.006*"witness" + 0.005*"security" + 0.005*"iron"'),
 (2,
  '0.009*"campaign" + 0.008*"european" + 0.008*"françois" + 0.007*"vietnam" + 0.007*"al" + 0.006*"shanghai" + 0.006*"nun" + 0.006*"neither" + 0.006*"wanders" + 0.005*"naples"'),
 (3,
  '0.006*"folk" + 0.006*"ravaged" + 0.006*"deal" + 0.005*"ruin" + 0.005*"project" + 0.005*"frank" + 0.005*"florence" + 0.005*"suspect" + 0.005*"cancer" + 0.005*"kid"'),
 (4,
  '0.008*"lake" + 0.008*"rule" + 0.007*"charge" + 0.007*"mayor" + 0.006*"quiet" + 0.006*"stolen" + 0.006*"sadistic" + 0.006*"presence" + 0.006*"mile" + 0.006*"superior"'),
 (5,
  '0.012*"bill" + 0.009*"sarah" + 0.008*"stone" + 0.007*"peasant" + 0.007*"musician" + 0.007*"ghost" + 0.007*

In [11]:
for i in range(15):
    print('Terms: ' + ', '.join([term for term, _ in lda.show_topic(i)]))
    print()

Terms: war, leo, story, film, woman, take, get, go, find, help

Terms: father, mother, child, find, work, son, de, new, house, death

Terms: year, take, pierre, time, film, woman, father, part, find, get

Terms: film, day, daughter, play, woman, world, work, relationship, get, find

Terms: war, woman, story, find, world, wife, time, soldier, take, husband

Terms: woman, story, friend, film, go, time, new, find, year, relationship

Terms: brother, day, film, story, try, see, get, wife, travel, based

Terms: town, job, year, anna, meet, find, get, friend, take, woman

Terms: year, son, father, three, woman, mother, friend, time, living, meet

Terms: year, wife, son, story, find, work, come, day, mother, father

Terms: new, begin, find, year, father, becomes, set, kill, must, max

Terms: story, find, year, job, child, woman, time, becomes, way, get

Terms: find, son, new, take, go, war, wife, day, world, men

Terms: meet, mother, night, try, becomes, soon, movie, fall, day, girl

Terms: g